In [2]:
%pylab inline
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import matplotlib.pyplot as plt
from os import listdir

Populating the interactive namespace from numpy and matplotlib


In [3]:
cd C:\LeiLin\USX28452\Downloads\bike_demand_prediction

C:\LeiLin\USX28452\Downloads\bike_demand_prediction


In [4]:
bikedf = pd.read_csv('NYC_hourly_bike_sharing_201307_201606.csv')

In [5]:
weatherdf = pd.read_csv('hourly_weather_20100101_20160809.csv')

In [6]:
bikedf = pd.merge(bikedf, weatherdf, how = 'left', on = ['year', 'month', 'day', 'hourofday','dayofweek'])

In [7]:
bikedf.head()

,Unnamed: 0_x,day,dayofweek,hourofday,month,year,Customer,Subscriber,Unnamed: 0_y,TemperatureF_x,Dew PointF_x,Humidity_x,Sea Level PressureIn_x,VisibilityMPH_x,Wind SpeedMPH_x,PrecipitationIn_x
0,0,1,0,0,7,2013,90.0,139.0,30648,75.0,70.0,84.0,29.930,8.0,1.75,0.0
1,1,1,0,1,7,2013,35.0,71.0,30649,75.0,70.0,84.0,29.935,8.0,2.30,0.0
2,2,1,0,2,7,2013,34.0,40.0,30650,73.9,70.0,87.0,29.910,8.0,3.50,0.0
3,3,1,0,3,7,2013,21.0,20.0,30651,73.9,70.0,87.0,29.910,9.0,0.00,0.0
4,4,1,0,4,7,2013,14.0,23.0,30652,73.9,70.0,87.0,29.910,9.0,0.00,0.0


In [23]:
def get_rmsle(y_pred, y_actual):
    diff = y_pred - y_actual
    mean_error = np.square(diff).mean()
    return np.sqrt(mean_error)

def get_mape(y_pred, y_actual):
    diff = abs(y_pred - y_actual)
    diff_per = diff[y_actual > 0] / y_actual[y_actual > 0]
    return np.mean(diff_per)

def custom_train_test_split(data, cutoff_day=25):
    train = data[data['day'] <= cutoff_day]
    test = data[data['day'] > cutoff_day]

    return train, test

def prep_data(data, input_cols):
    X = data[input_cols].as_matrix()
    y_c = data['Customer'].as_matrix()
    y_s = data['Subscriber'].as_matrix()

    return X, y_c, y_s

def predict_on_validation_set(model, input_cols):

    train, test = custom_train_test_split(bikedf) #bikedf

    X_train, y_train_cus, y_train_sub = prep_data(train, input_cols)
    X_test, y_test_cus, y_test_sub = prep_data(test, input_cols)

    model_cus = model.fit(X_train, y_train_cus)
    y_pred_cus = model_cus.predict(X_test)

    model_sub = model.fit(X_train, y_train_sub)
    y_pred_sub = model_sub.predict(X_test)

    y_pred_comb = np.round(y_pred_cus + y_pred_sub)
    y_pred_comb[y_pred_comb < 0] = 0
    
    # calculate the groundtruth
    y_test_comb = y_test_cus + y_test_sub

    score = get_rmsle(y_pred_comb, y_test_comb)
    score2 = get_mape(y_pred_comb, y_test_comb)
    
    return (y_pred_comb, y_test_comb, score, score2)

In [24]:
#n_estimators:number of trees 1000-324.13
# min_sample_leaf：minimum leaf size
# n_jobs: how many processor it allowed to use, -1 means no restriction
# random_state：random seed

params = {'n_estimators': 500, 'min_samples_leaf': 5, 'random_state': 0, 'min_samples_split' : 5, 'n_jobs': -1}
rf_model = RandomForestRegressor(**params)
rf_cols = [
    'year', 'month', 'day', 'hourofday','dayofweek',
    'TemperatureF_x','Dew PointF_x','Humidity_x','Sea Level PressureIn_x','VisibilityMPH_x','Wind SpeedMPH_x',
    'PrecipitationIn_x'
    ]

(rf_p, rf_t, rf_score, rf_mape) = predict_on_validation_set(rf_model, rf_cols)
print (rf_score)
print (rf_mape)

341.23953501
0.726821824384


In [29]:
params = {'n_estimators': 150, 'max_depth': 5, 'random_state': 0, 'min_samples_leaf' : 10, 'learning_rate': 0.1, 'subsample': 0.7, 'loss': 'ls'}
gbm_model = GradientBoostingRegressor(**params)
gbm_cols = [
    'year', 'month', 'day', 'hourofday','dayofweek',
    'TemperatureF_x','Dew PointF_x','Humidity_x','Sea Level PressureIn_x','VisibilityMPH_x','Wind SpeedMPH_x',
    'PrecipitationIn_x'
]

(gbm_p, gbm_t, gbm_score) = predict_on_validation_set(gbm_model, gbm_cols)
print (gbm_score)

313.233575818
